<a href="https://colab.research.google.com/github/you0229/kaggle_titanic/blob/master/main_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
# libraries
import pandas as pd
import numpy as np
from keras import models
from keras import layers
from keras.utils import to_categorical
import matplotlib.pyplot as plt
# %matplotlib inline

COLAB_FLG = 1
TRAIN_PATH = '/kaggle/input/titanic/train.csv'
TEST_PATH = '/kaggle/input/titanic/test.csv'
COLAB_TRAIN_PATH = '/content/drive/My Drive/MachineLeaning/kaggle_titanic/train.csv'
COLAB_TEST_PATH = '/content/drive/My Drive/MachineLeaning/kaggle_titanic/test.csv'

# データ読み込み
def read_data():
  ## 読み込むデータのパス切り替え
  if COLAB_FLG:
    TRAIN_PATH = COLAB_TRAIN_PATH
    TEST_PATH = COLAB_TEST_PATH
  
  print('Train data reading...')
  train = pd.read_csv(TRAIN_PATH)
  print("Train data is column {}, rows: {}".format(train.shape[0], train.shape[1]))
  print('Test data reading...')
  test = pd.read_csv(TEST_PATH)
  print("Test data is column {}, rows: {}".format(test.shape[0], test.shape[1]))

  return train, test

# データ整理
def pre_processing(train, test):
  # train と test を合わせて前処理
  print('Combine train and test')
  train['is_train'] = 1
  test['is_train'] = 0
  train_test = pd.concat([train.drop('Survived', axis=1),test], axis=0)

  # Sex
  print('Modifying Sex column')
  train_test = train_test.replace('female', 0).replace('male', 1)

  # Ticket
  print('Modifying Ticket column')
  ticket_to_num = {'A':0, 'P':1, 'S':2, '1':3, '3':4, '2':5, 'C':6, '7':7, 'W':8, '4':9, 'F':9, 'L':10, '9':11,'6':12, '5':13, '8':14}
  train_test['Ticket_Left'] = train_test['Ticket'].apply(lambda x: str(x)[0])
  train_test['Ticket_Left'] = train_test['Ticket_Left'].replace(ticket_to_num)
  train_test = train_test.drop('Ticket', axis=1)

  # Embarked
  print('Modifying Embarked column')
  embarked_to_num = {'S':0, 'C':1, 'Q':2}
  train_test['Embarked'] = train_test['Embarked'].replace(embarked_to_num)
  train_test['Embarked'] = train_test['Embarked'].fillna(train_test['Embarked'].mean())
  train_test = train_test.drop('Embarked', axis=1)

  # Cabin
  # 204/891 したデータが入っていないため削除
  print('Modifying Cabin column')
  train_test = train_test.drop('Cabin', axis=1)

  # Age
  # print('Modifying Age column')
  # train_test = train_test.dropna(subset=['Age'])

  # Name
  # 関係なさそうだから削除
  print('Modifying Name column')
  train_test = train_test.drop('Name', axis=1)

  # Fare
  # print('Modifying Fare column')
  # train_test = train_test.dropna(subset=['Fare'])

  # train test
  print('Split train and test')
  train_result = train_test.loc[train_test['is_train'] == 1]
  test_result = train_test.loc[train_test['is_train'] == 0]

  train_result = train_result.drop('is_train', axis=1)
  test_result = test_result.drop('is_train', axis=1)

  # add Survived
  train_result['Survived'] = train['Survived']

  # Delete Age and Fare
  train_result = train_result.dropna(subset=['Age', 'Fare'])
  test_result = test_result.fillna(test_result.mean())

  return train_result, test_result

# モデル作成
def create_model(x):
  network = models.Sequential()
  network.add(layers.Dense(512, activation='relu', input_shape=(x.shape[1],)))
  network.add(layers.Dense(10,activation='softmax'))
  network.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

  return network


train, test = read_data()

train, test = pre_processing(train, test)

columns = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Ticket_Left']

train_data = train[columns].values
train_lavels = train['Survived'].values

x_train = np.asarray(train_data).astype('float32')
y_train = np.asarray(train_lavels).astype('float32')

test_data = test[columns].values.astype('float32')


#正規化
for i in range(len(columns)-1):
    mean = x_train.mean(axis=0)[i]
    std = x_train.std(axis=0)[i]

    x_train[:, i] = (x_train[:, i] - mean) / std
    test_data[:, i] = (test_data[:, i] - mean) / std

# create model
model = models.Sequential()
model.add(layers.Dense(8, activation='relu', input_shape=(len(columns),)))
model.add(layers.Dense(1,activation='softmax'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# fitting
model.fit(x_train, y_train, epochs=3, batch_size=100)

train_loss, train_acc = model.evaluate(x_train, y_train)
print('train_acc : ', train_acc)

Y_pred = model.predict(test_data)

import csv
with open("predict_result_data.csv", "w") as f:
    writer = csv.writer(f, lineterminator='\n')
    writer.writerow(["PassengerId", "Survived"])
    for pid, survived in zip(test['PassengerId'].astype(int), Y_pred[:, 0].astype(int)):
        writer.writerow([pid, survived])

Train data reading...
Train data is column 891, rows: 12
Test data reading...
Test data is column 418, rows: 11
Combine train and test
Modifying Sex column
Modifying Ticket column
Modifying Embarked column
Modifying Cabin column
Modifying Name column
Split train and test





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/3





714/714 [==============================] - 1s 836us/step - loss: 9.4672 - acc: 0.4062
Epoch 2/3
714/714 [==============================] - 0s 17us/step - loss: 9.4672 - acc: 0.4062
Epoch 3/3
714/714 [==============================] - 0s 48us/step
train_acc :  0.4061624651529542
